In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19  # VGG19 model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle

# Define the paths
augmented_train_dir = r'C:\Users\kaush\Downloads\indian skins scrapped\augmented_train'  # Directory containing all data
test_dir = r'C:\Users\kaush\Downloads\indian skins scrapped\train\testing'  # Path for test data

# Image data generator for preprocessing (normalization only) with 80-20 split
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    validation_split=0.20  # 20% validation split
)

# Load the training data (80% of the data)
train_data = datagen.flow_from_directory(
    augmented_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # Use 'training' subset (80%)
    shuffle=True
)

# Load the validation data (20% of the data)
val_data = datagen.flow_from_directory(
    augmented_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Use 'validation' subset (20%)
    shuffle=True
)

# Load the test data (not augmented)
test_data = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Build the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),  # Flatten the output from the convolutional base
    Dropout(0.5),  # Add dropout to reduce overfitting
    Dense(3, activation='softmax')  # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Save the best model during training
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Adjust epochs as needed
    callbacks=[checkpoint]
)

# Save the training history
with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

# Function to print precision, recall, and F1 score
def print_classification_metrics(data, dataset_name):
    data.reset()  # Reset data generator
    predictions = model.predict(data)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = data.classes
    
    # Calculate precision, recall, and F1 score
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')
    
    # Print the metrics
    print(f"\n{dataset_name} Metrics:")
    print(f"Precision = {precision * 100:.2f}")
    print(f"Recall = {recall * 100:.2f}")
    print(f"F1 Score = {f1 * 100:.2f}")

# Print precision, recall, and F1 score for validation set
print_classification_metrics(val_data, "Validation Set")

# Print precision, recall, and F1 score for test set
print_classification_metrics(test_data, "Test Set")

# Print the training and validation accuracy and loss
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]

print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Training Loss: {train_loss:.4f}")
print(f"Validation Loss: {val_loss:.4f}")


Found 521 images belonging to 3 classes.
Found 129 images belonging to 3 classes.
Found 17 images belonging to 3 classes.
Epoch 1/20


C:\Users\kaush\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2890 - loss: 1.6339
Epoch 1: val_accuracy improved from -inf to 0.37209, saving model to best_model.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.2915 - loss: 1.6219 - val_accuracy: 0.3721 - val_loss: 1.1553
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3809 - loss: 1.2405
Epoch 2: val_accuracy improved from 0.37209 to 0.41860, saving model to best_model.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.3808 - loss: 1.2402 - val_accuracy: 0.4186 - val_loss: 1.0857
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4452 - loss: 1.1347
Epoch 3: val_accuracy improved from 0.41860 to 0.46512, saving model to best_model.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.4441 - loss: 1.1359 - val_accuracy: 0.4651 - val_loss: 1.0589
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4165 - loss: 1.1950
Epoch 4: val_accuracy improved from 0.46512 to 0.49612, saving model t